In [2]:
import albumentations as A
import timm
import torch
from addict import Dict
from main_pipeline import main
from utils import LabelSmoothingCrossEntropy
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau
from trainer import train_step, val_step
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from dataset import CarDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
train_aug = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CLAHE(p=0.5),
        A.ChannelShuffle(p=0.5),
        A.ColorJitter(p=0.5),
        A.ChannelDropout(p=0.5),
        A.RandomGamma(p=0.5),
        A.RGBShift(p=0.5),
        A.GaussNoise(p=0.5),
        A.HueSaturationValue(p=0.5),
    ]
)

## База на mps

### VAL_ACC = 0.89
### PUBLIC = 0.8898

In [ ]:
args = Dict()
args.batch_size = 32
args.lr = 1e-3
args.w_d = 5e-4
args.phase = "train"
args.img_size = 224
args.device = "mps"
args.model_path = "../models/efficientnetv2_2"

In [ ]:
model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.4)
model.reset_classifier(196)
args.model = model

In [ ]:
args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
args.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    args.optimizer, mode="max", patience=3, min_lr=1e-6, verbose=True
)

In [ ]:
args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

In [ ]:
main(args, trainsforms=train_aug)

## Обучение на GPU

In [3]:
args = Dict()
args.batch_size = 64
args.lr = 1e-3
args.w_d = 5e-4
args.phase = "train"
args.img_size = 320
args.device = "cuda"
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_gpu.pth"
args.epochs = None

In [4]:
model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.7)
model.reset_classifier(196)
args.model = model

In [5]:
args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
args.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    args.optimizer, mode="max", patience=3, min_lr=1e-6, verbose=True
)
args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

In [6]:
main(args, trainsforms=train_aug)

0:00:14.368214 Epoch: [1][1/102]	Loss 5.7968750	Prec@1 0.000
0:00:15.193479 Epoch: [1][2/102]	Loss 5.7460938	Prec@1 0.000
0:00:15.747359 Epoch: [1][3/102]	Loss 5.7395833	Prec@1 0.000
0:00:16.308494 Epoch: [1][4/102]	Loss 5.6787109	Prec@1 0.004
0:00:16.861562 Epoch: [1][5/102]	Loss 5.6710937	Prec@1 0.003
0:00:17.428031 Epoch: [1][6/102]	Loss 5.6907552	Prec@1 0.003
0:00:17.995056 Epoch: [1][7/102]	Loss 5.6830357	Prec@1 0.002
0:00:18.580585 Epoch: [1][8/102]	Loss 5.6972656	Prec@1 0.002
0:00:19.159328 Epoch: [1][9/102]	Loss 5.6909722	Prec@1 0.002
0:00:20.637339 Epoch: [1][10/102]	Loss 5.6863281	Prec@1 0.002
0:00:26.077713 Epoch: [1][11/102]	Loss 5.6800426	Prec@1 0.001
0:00:26.639093 Epoch: [1][12/102]	Loss 5.6585286	Prec@1 0.003
0:00:27.242614 Epoch: [1][13/102]	Loss 5.6475361	Prec@1 0.004
0:00:27.873356 Epoch: [1][14/102]	Loss 5.6403460	Prec@1 0.003
0:00:31.794567 Epoch: [1][15/102]	Loss 5.6231771	Prec@1 0.004
0:00:32.372368 Epoch: [1][16/102]	Loss 5.6115723	Prec@1 0.005
0:00:32.962349 Ep

KeyboardInterrupt: 

### VAL_ACC = 0.909
### PUBLIC = 0.91069

## Dropout, в прошлый раз была слишком сильная регуляризация, хочется быстрее обучаться

In [4]:
args = Dict()
args.batch_size = 64
args.lr = 1e-3
args.w_d = 5e-4
args.phase = "train"
args.img_size = 320
args.device = "cuda"
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_dropout.pth"

In [5]:
model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.5)
model.reset_classifier(196)
args.model = model

In [6]:
args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
args.lr_scheduler = ReduceLROnPlateau(args.optimizer, mode="max", patience=3, min_lr=1e-6, verbose=True)
args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

In [7]:
main(args, trainsforms=train_aug)

0:00:16.516084 Epoch: [1][1/102]	Loss 5.6093750	Prec@1 0.016
0:00:17.107525 Epoch: [1][2/102]	Loss 5.5957031	Prec@1 0.008
0:00:17.669519 Epoch: [1][3/102]	Loss 5.5677083	Prec@1 0.010
0:00:18.249668 Epoch: [1][4/102]	Loss 5.5507812	Prec@1 0.008
0:00:18.867810 Epoch: [1][5/102]	Loss 5.5796875	Prec@1 0.006
0:00:19.440149 Epoch: [1][6/102]	Loss 5.5677083	Prec@1 0.008
0:00:20.010279 Epoch: [1][7/102]	Loss 5.5714286	Prec@1 0.009
0:00:20.585956 Epoch: [1][8/102]	Loss 5.5629883	Prec@1 0.008
0:00:21.165185 Epoch: [1][9/102]	Loss 5.5577257	Prec@1 0.010
0:00:23.273363 Epoch: [1][10/102]	Loss 5.5574219	Prec@1 0.011
0:00:23.892649 Epoch: [1][11/102]	Loss 5.5678267	Prec@1 0.010
0:00:24.486510 Epoch: [1][12/102]	Loss 5.5576172	Prec@1 0.009
0:00:25.110677 Epoch: [1][13/102]	Loss 5.5588942	Prec@1 0.008
0:00:29.424167 Epoch: [1][14/102]	Loss 5.5485491	Prec@1 0.008
0:00:32.514588 Epoch: [1][15/102]	Loss 5.5416667	Prec@1 0.008
0:00:33.108453 Epoch: [1][16/102]	Loss 5.5329590	Prec@1 0.008
0:00:33.718448 Ep

KeyboardInterrupt: 

### VAL_ACC = 0.916
### PUBLIC = 0.92114

## Learning rate, другой scheduler

In [4]:
args = Dict()
args.batch_size = 64
args.lr = 1e-3
args.w_d = 5e-4
args.phase = "train"
args.img_size = 320
args.device = "cuda"
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_lr.pth"
args.epochs = 40

In [5]:
model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.5)
model.reset_classifier(196)
args.model = model

In [6]:
args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
args.lr_scheduler = OneCycleLR(args.optimizer, max_lr=0.01, steps_per_epoch=102, epochs=20, verbose=True)
args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

Adjusting learning rate of group 0 to 4.0000e-04.


In [7]:
main(args, trainsforms=train_aug)

0:00:15.894708 Epoch: [1][1/102]	Loss 5.4218750	Prec@1 0.047
0:00:16.538131 Epoch: [1][2/102]	Loss 5.5078125	Prec@1 0.023
0:00:17.101221 Epoch: [1][3/102]	Loss 5.5833333	Prec@1 0.016
0:00:17.689047 Epoch: [1][4/102]	Loss 5.5820312	Prec@1 0.012
0:00:18.300015 Epoch: [1][5/102]	Loss 5.5945313	Prec@1 0.009
0:00:18.900333 Epoch: [1][6/102]	Loss 5.5904948	Prec@1 0.008
0:00:19.513035 Epoch: [1][7/102]	Loss 5.5848214	Prec@1 0.007
0:00:20.125610 Epoch: [1][8/102]	Loss 5.5776367	Prec@1 0.006
0:00:22.189580 Epoch: [1][9/102]	Loss 5.5633681	Prec@1 0.005
0:00:23.869397 Epoch: [1][10/102]	Loss 5.5609375	Prec@1 0.005
0:00:24.443020 Epoch: [1][11/102]	Loss 5.5607244	Prec@1 0.007
0:00:25.016196 Epoch: [1][12/102]	Loss 5.5605469	Prec@1 0.007
0:00:29.849996 Epoch: [1][13/102]	Loss 5.5597957	Prec@1 0.006
0:00:31.455778 Epoch: [1][14/102]	Loss 5.5485491	Prec@1 0.006
0:00:32.037835 Epoch: [1][15/102]	Loss 5.5427083	Prec@1 0.005
0:00:32.605179 Epoch: [1][16/102]	Loss 5.5358887	Prec@1 0.006
0:00:38.520432 Ep

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


0:00:11.566978 Epoch: [2][1/102]	Loss 4.4375000	Prec@1 0.219
0:00:12.153217 Epoch: [2][2/102]	Loss 4.5078125	Prec@1 0.156
0:00:12.726990 Epoch: [2][3/102]	Loss 4.4674479	Prec@1 0.172
0:00:13.327324 Epoch: [2][4/102]	Loss 4.4345703	Prec@1 0.164
0:00:19.522814 Epoch: [2][5/102]	Loss 4.4171875	Prec@1 0.178
0:00:20.099380 Epoch: [2][6/102]	Loss 4.4231771	Prec@1 0.167
0:00:20.697821 Epoch: [2][7/102]	Loss 4.4118304	Prec@1 0.170
0:00:21.295449 Epoch: [2][8/102]	Loss 4.3984375	Prec@1 0.178
0:00:23.008528 Epoch: [2][9/102]	Loss 4.3932292	Prec@1 0.182
0:00:23.608889 Epoch: [2][10/102]	Loss 4.3777344	Prec@1 0.183
0:00:24.209206 Epoch: [2][11/102]	Loss 4.3497869	Prec@1 0.193
0:00:24.823311 Epoch: [2][12/102]	Loss 4.3525391	Prec@1 0.195
0:00:29.506163 Epoch: [2][13/102]	Loss 4.3479567	Prec@1 0.196
0:00:30.138911 Epoch: [2][14/102]	Loss 4.3236607	Prec@1 0.202
0:00:30.789966 Epoch: [2][15/102]	Loss 4.3049479	Prec@1 0.206
0:00:31.406122 Epoch: [2][16/102]	Loss 4.3012695	Prec@1 0.204
0:00:36.919284 Ep

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


0:00:06.702057 Epoch: [3][1/102]	Loss 2.6699219	Prec@1 0.734
0:00:07.285710 Epoch: [3][2/102]	Loss 2.6269531	Prec@1 0.727
0:00:09.199398 Epoch: [3][3/102]	Loss 2.5976562	Prec@1 0.729
0:00:09.779045 Epoch: [3][4/102]	Loss 2.6342773	Prec@1 0.723
0:00:17.405887 Epoch: [3][5/102]	Loss 2.6269531	Prec@1 0.728
0:00:17.997900 Epoch: [3][6/102]	Loss 2.6279297	Prec@1 0.727
0:00:18.589454 Epoch: [3][7/102]	Loss 2.6143973	Prec@1 0.734
0:00:19.207186 Epoch: [3][8/102]	Loss 2.6127930	Prec@1 0.715
0:00:23.762714 Epoch: [3][9/102]	Loss 2.6132812	Prec@1 0.708
0:00:24.388851 Epoch: [3][10/102]	Loss 2.6074219	Prec@1 0.705
0:00:24.988294 Epoch: [3][11/102]	Loss 2.6019176	Prec@1 0.712
0:00:25.591364 Epoch: [3][12/102]	Loss 2.6082357	Prec@1 0.711
0:00:30.183193 Epoch: [3][13/102]	Loss 2.5994591	Prec@1 0.715
0:00:30.764492 Epoch: [3][14/102]	Loss 2.5912388	Prec@1 0.720
0:00:31.417761 Epoch: [3][15/102]	Loss 2.5930990	Prec@1 0.719
0:00:32.007803 Epoch: [3][16/102]	Loss 2.5886230	Prec@1 0.718
0:00:34.811442 Ep

...

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


0:00:07.728017 Epoch: [21][1/102]	Loss 1.6933594	Prec@1 0.984
0:00:12.266976 Epoch: [21][2/102]	Loss 1.6738281	Prec@1 0.992
0:00:12.865001 Epoch: [21][3/102]	Loss 1.6816406	Prec@1 0.984
0:00:13.452205 Epoch: [21][4/102]	Loss 1.6777344	Prec@1 0.988
0:00:17.155777 Epoch: [21][5/102]	Loss 1.6724609	Prec@1 0.991
0:00:21.188782 Epoch: [21][6/102]	Loss 1.6679688	Prec@1 0.992
0:00:21.765542 Epoch: [21][7/102]	Loss 1.6808036	Prec@1 0.987
0:00:22.377497 Epoch: [21][8/102]	Loss 1.6791992	Prec@1 0.986
0:00:25.697794 Epoch: [21][9/102]	Loss 1.6830512	Prec@1 0.984
0:00:26.813132 Epoch: [21][10/102]	Loss 1.6862305	Prec@1 0.983
0:00:27.397372 Epoch: [21][11/102]	Loss 1.6883878	Prec@1 0.984
0:00:27.997206 Epoch: [21][12/102]	Loss 1.6859538	Prec@1 0.984
0:00:31.968080 Epoch: [21][13/102]	Loss 1.6831430	Prec@1 0.986
0:00:33.421803 Epoch: [21][14/102]	Loss 1.6833845	Prec@1 0.984
0:00:34.005338 Epoch: [21][15/102]	Loss 1.6884115	Prec@1 0.983
0:00:34.630606 Epoch: [21][16/102]	Loss 1.6865234	Prec@1 0.983
0

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


0:00:07.421303 Epoch: [22][1/102]	Loss 1.6533203	Prec@1 1.000
0:00:08.026424 Epoch: [22][2/102]	Loss 1.6748047	Prec@1 1.000
0:00:10.238267 Epoch: [22][3/102]	Loss 1.6809896	Prec@1 0.995
0:00:10.852242 Epoch: [22][4/102]	Loss 1.6845703	Prec@1 0.992
0:00:16.948887 Epoch: [22][5/102]	Loss 1.6783203	Prec@1 0.994
0:00:17.520219 Epoch: [22][6/102]	Loss 1.6765951	Prec@1 0.992
0:00:18.096783 Epoch: [22][7/102]	Loss 1.6745257	Prec@1 0.993
0:00:18.696649 Epoch: [22][8/102]	Loss 1.6761475	Prec@1 0.992
0:00:22.650273 Epoch: [22][9/102]	Loss 1.6740451	Prec@1 0.993
0:00:23.268423 Epoch: [22][10/102]	Loss 1.6705078	Prec@1 0.994
0:00:23.856240 Epoch: [22][11/102]	Loss 1.6674361	Prec@1 0.994
0:00:24.468824 Epoch: [22][12/102]	Loss 1.6649577	Prec@1 0.995
0:00:29.058515 Epoch: [22][13/102]	Loss 1.6653395	Prec@1 0.994
0:00:29.649246 Epoch: [22][14/102]	Loss 1.6690848	Prec@1 0.993
0:00:31.300220 Epoch: [22][15/102]	Loss 1.6685547	Prec@1 0.993
0:00:31.913254 Epoch: [22][16/102]	Loss 1.6731567	Prec@1 0.990
0

KeyboardInterrupt: 

### VAL_ACC = 0.886
### PUBLIC = 0.87139

## L2 - regulisation сделаем больше

In [5]:
args = Dict()
args.batch_size = 64
args.lr = 1e-3
args.w_d = 5e-3
args.phase = "train"
args.img_size = 320
args.device = "cuda"
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_l2.pth"
args.epochs = 100

In [6]:
model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.5)
model.reset_classifier(196)
args.model = model

In [7]:
args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
args.lr_scheduler = ReduceLROnPlateau(args.optimizer, mode="max", patience=3, min_lr=1e-6, verbose=True)
args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

In [ ]:
main(args, trainsforms=train_aug)

0:00:14.321706 Epoch: [1][1/102]	Loss 5.5507812	Prec@1 0.000
0:00:14.920521 Epoch: [1][2/102]	Loss 5.5410156	Prec@1 0.000
0:00:15.491281 Epoch: [1][3/102]	Loss 5.5559896	Prec@1 0.000
0:00:16.101102 Epoch: [1][4/102]	Loss 5.5664062	Prec@1 0.004
0:00:16.703328 Epoch: [1][5/102]	Loss 5.5812500	Prec@1 0.006
0:00:21.946001 Epoch: [1][6/102]	Loss 5.5937500	Prec@1 0.005
0:00:22.516462 Epoch: [1][7/102]	Loss 5.5926339	Prec@1 0.007
0:00:23.123558 Epoch: [1][8/102]	Loss 5.5722656	Prec@1 0.010
0:00:23.719496 Epoch: [1][9/102]	Loss 5.5824653	Prec@1 0.009
0:00:26.754012 Epoch: [1][10/102]	Loss 5.5675781	Prec@1 0.011
0:00:27.353052 Epoch: [1][11/102]	Loss 5.5727983	Prec@1 0.010
0:00:27.983559 Epoch: [1][12/102]	Loss 5.5696615	Prec@1 0.009
0:00:28.594862 Epoch: [1][13/102]	Loss 5.5712139	Prec@1 0.010
0:00:31.746823 Epoch: [1][14/102]	Loss 5.5574777	Prec@1 0.010
0:00:32.345008 Epoch: [1][15/102]	Loss 5.5505208	Prec@1 0.009
0:00:32.929606 Epoch: [1][16/102]	Loss 5.5380859	Prec@1 0.010
0:00:33.510423 Ep

### VAL_ACC = 0.913
### PUBLIC = 0.91666

### В целом ничего глобально не поменялось, только обучаться стало чуть хуже

## L2 - regulisation сделаем еще больше

In [4]:
args = Dict()
args.batch_size = 64
args.lr = 1e-3
args.w_d = 5e-2
args.phase = "train"
args.img_size = 320
args.device = "cuda"
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_l2_more.pth"
args.epochs = 100

In [5]:
model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.5)
model.reset_classifier(196)
args.model = model

In [6]:
args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
args.lr_scheduler = ReduceLROnPlateau(args.optimizer, mode="max", patience=3, min_lr=1e-6, verbose=True)
args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

In [ ]:
main(args, trainsforms=train_aug)

0:00:14.854387 Epoch: [1][1/102]	Loss 5.5078125	Prec@1 0.000
0:00:15.580891 Epoch: [1][2/102]	Loss 5.5332031	Prec@1 0.000
0:00:16.182558 Epoch: [1][3/102]	Loss 5.5924479	Prec@1 0.000
0:00:16.795485 Epoch: [1][4/102]	Loss 5.5722656	Prec@1 0.000
0:00:17.405844 Epoch: [1][5/102]	Loss 5.5914063	Prec@1 0.000
0:00:18.035610 Epoch: [1][6/102]	Loss 5.5846354	Prec@1 0.003
0:00:18.631078 Epoch: [1][7/102]	Loss 5.5742188	Prec@1 0.007
0:00:19.246889 Epoch: [1][8/102]	Loss 5.5727539	Prec@1 0.006
0:00:19.835948 Epoch: [1][9/102]	Loss 5.5720486	Prec@1 0.007
0:00:25.575211 Epoch: [1][10/102]	Loss 5.5648437	Prec@1 0.006
0:00:26.160565 Epoch: [1][11/102]	Loss 5.5692472	Prec@1 0.006
0:00:26.749716 Epoch: [1][12/102]	Loss 5.5611979	Prec@1 0.007
0:00:27.324490 Epoch: [1][13/102]	Loss 5.5573918	Prec@1 0.006
0:00:30.040407 Epoch: [1][14/102]	Loss 5.5477121	Prec@1 0.006
0:00:30.863700 Epoch: [1][15/102]	Loss 5.5419271	Prec@1 0.005
0:00:31.478363 Epoch: [1][16/102]	Loss 5.5317383	Prec@1 0.005
0:00:32.094807 Ep

KeyboardInterrupt: 

### VAL_ACC = 0.907
### PUBLIC = 0.91492

### Ну короче не помогло

## Еще аугментации

In [4]:
my_aug = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CLAHE(p=0.5),
        A.ChannelShuffle(p=0.5),
        A.ColorJitter(p=0.5),
        A.ChannelDropout(p=0.5),
        A.RandomGamma(p=0.5),
        A.RGBShift(p=0.5),
        A.GaussNoise(p=0.5),
        A.HueSaturationValue(p=0.5),

        # A.RandomCropFromBorders(p=0.5, crop_left=0.1, crop_right=0.1, crop_top=0.1, crop_bottom=0.1),
        A.CoarseDropout(p=0.5, min_holes=8, max_holes=16),
        A.Downscale(p=0.5, scale_min=0.7, scale_max=0.99),
        A.Equalize(p=0.5),
        A.ISONoise(p=0.5),
        A.MultiplicativeNoise(p=0.5),
        A.PixelDropout(p=0.5, dropout_prob=0.1),
        A.SafeRotate(p=0.5, limit=(-15, 15)),

        # A.Emboss(p=0.5),
        # A.FancyPCA(p=0.5),
        # A.RandomToneCurve(p=0.5),
    ]
)

In [5]:
args = Dict()
args.batch_size = 64
args.lr = 1e-3
args.w_d = 5e-4
args.phase = "train"
args.img_size = 320
args.device = "cuda"
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_aug.pth"
args.epochs = 100

In [6]:
model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.5)
model.reset_classifier(196)
args.model = model

In [7]:
args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
args.lr_scheduler = ReduceLROnPlateau(args.optimizer, mode="max", patience=3, min_lr=1e-6, verbose=True)
args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

In [8]:
main(args, trainsforms=my_aug)

0:00:25.331640 Epoch: [1][1/102]	Loss 5.4375000	Prec@1 0.016
0:00:25.940749 Epoch: [1][2/102]	Loss 5.4296875	Prec@1 0.023
0:00:26.497470 Epoch: [1][3/102]	Loss 5.4817708	Prec@1 0.021
0:00:27.074033 Epoch: [1][4/102]	Loss 5.5068359	Prec@1 0.020
0:00:27.650728 Epoch: [1][5/102]	Loss 5.5085938	Prec@1 0.019
0:00:28.249445 Epoch: [1][6/102]	Loss 5.5338542	Prec@1 0.016
0:00:28.810400 Epoch: [1][7/102]	Loss 5.5502232	Prec@1 0.018
0:00:29.393601 Epoch: [1][8/102]	Loss 5.5556641	Prec@1 0.018
0:00:34.437985 Epoch: [1][9/102]	Loss 5.5564236	Prec@1 0.019
0:00:45.800391 Epoch: [1][10/102]	Loss 5.5800781	Prec@1 0.017
0:00:46.360195 Epoch: [1][11/102]	Loss 5.5898438	Prec@1 0.016
0:00:46.942429 Epoch: [1][12/102]	Loss 5.5800781	Prec@1 0.014
0:00:47.547001 Epoch: [1][13/102]	Loss 5.5778245	Prec@1 0.013
0:00:55.760577 Epoch: [1][14/102]	Loss 5.5747768	Prec@1 0.013
0:00:56.317185 Epoch: [1][15/102]	Loss 5.5635417	Prec@1 0.016
0:00:56.879826 Epoch: [1][16/102]	Loss 5.5576172	Prec@1 0.015
0:00:57.465744 Ep

KeyboardInterrupt: 

### В целом помогло, но только в начале. Возможно большее количество аугментаций бы помогло, но это уже очень долго. Или стоит рассмотреть архитектуру побольше...

## TTA

In [10]:
model = timm.create_model('tf_efficientnetv2_s.in21k', pretrained=True)
model.reset_classifier(196)

args = Dict()
args.device = "cuda"
args.img_size = 320
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_dropout.pth"
args.output_file = "/home/jupyter/datasphere/project/models/efficientnetv2_dropout_sub.csv"

test_dataset = CarDataset(df_filename="/home/jupyter/mnt/datasets/intro-dl-2024/intro-dl-2024/sample_submission.csv",
                          phase="test",
                          dataroot="/home/jupyter/mnt/datasets/intro-dl-2024/intro-dl-2024/",
                          img_size=args.img_size, transforms=trainsforms)

test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=4)

model.load_state_dict(torch.load(args.model_path))
model = model.to(args.device)
model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
   

In [ ]:
tta_num = 8
final_predictions = None
all_outputs = []
with torch.no_grad():
    for i in range(tta_num):
        temp_outputs = []
        for features, _ in tqdm(test_loader):
            outputs = model(features.to(args.device))
            temp_outputs.extend(outputs.cpu())
        all_outputs.append(torch.stack(temp_outputs))

In [ ]:
mean_outputs = torch.mean(torch.stack(all_outputs), dim=0)

In [ ]:
_, predicted = torch.max(mean_outputs.data, 1)
df = pd.read_csv('../../data/intro-dl-2024/sample_submission.csv')
df['label'] = predicted
df.to_csv(args.output_file, index=False)

### TTA не помог

## Итоговый вариант

In [ ]:
args = Dict()
args.batch_size = 64
args.lr = 1e-3
args.w_d = 5e-4
args.phase = "train"
args.img_size = 320
args.device = "cuda"
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_dropout.pth"

In [ ]:
model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.5)
model.reset_classifier(196)
args.model = model

In [ ]:
args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
args.lr_scheduler = ReduceLROnPlateau(args.optimizer, mode="max", patience=3, min_lr=1e-6, verbose=True)
args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

## CV-5 

In [4]:
transforms = train_aug

In [5]:
args = Dict()
args.batch_size = 64
args.lr = 1e-3
args.w_d = 5e-4
args.phase = "train"
args.img_size = 320
args.device = "cuda"
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_cv.pth"
args.epochs = 40

In [6]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
df = pd.read_csv('/home/jupyter/mnt/datasets/intro-dl-2024/intro-dl-2024/train.csv')

for fold, (train_idx, val_idx) in zip(range(1, 2), list(kf.split(df, df['label']))[1:2]):
    print(f"Training fold {fold + 1}")
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    
    model = timm.create_model("tf_efficientnetv2_s.in21k", pretrained=True, drop_rate=0.5)
    model.reset_classifier(196)
    args.model = model
    
    args.model_path = f"/home/jupyter/datasphere/project/models/efficientnetv2_cv{fold + 1}.pth"
    log_path = args.model_path[:-4]
    
    args.optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.w_d)
    args.lr_scheduler = ReduceLROnPlateau(args.optimizer, mode="max", patience=3, min_lr=1e-6, verbose=True)
    args.criterion = LabelSmoothingCrossEntropy(smoothing=0.2)

    train_dataset = CarDataset(df_filename=train_df,
                               phase=args.phase,
                               img_size=args.img_size,
                               transforms=transforms)

    val_dataset = CarDataset(df_filename=val_df,
                             phase=args.phase,
                             img_size=args.img_size)
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=True,
        num_workers=4, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=args.batch_size, shuffle=False,
        num_workers=4, pin_memory=True)
    
    scaler = torch.cuda.amp.GradScaler()
    device = args.device
    num_epoch = args.epochs if args.epochs is not None else 100
    model = model.to(device)
    
    best_val_acc = 0
    for epoch in range(1, 1 + num_epoch):
        train_step(train_loader, model, args.criterion, scaler, args.optimizer, device, epoch, log_path)

        val_accuracy = val_step(val_loader, model, args.criterion, device, log_path)
        args.lr_scheduler.step(val_accuracy)

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
        torch.save(model.state_dict(), args.model_path)

Training fold 2
0:00:14.965015 Epoch: [1][1/102]	Loss 5.4687500	Prec@1 0.000
0:00:15.584229 Epoch: [1][2/102]	Loss 5.4628906	Prec@1 0.008
0:00:16.179156 Epoch: [1][3/102]	Loss 5.5013021	Prec@1 0.005
0:00:16.772527 Epoch: [1][4/102]	Loss 5.5253906	Prec@1 0.004
0:00:17.384106 Epoch: [1][5/102]	Loss 5.5382812	Prec@1 0.006
0:00:17.992762 Epoch: [1][6/102]	Loss 5.5371094	Prec@1 0.005
0:00:18.576231 Epoch: [1][7/102]	Loss 5.5502232	Prec@1 0.004
0:00:19.202497 Epoch: [1][8/102]	Loss 5.5473633	Prec@1 0.006
0:00:21.838900 Epoch: [1][9/102]	Loss 5.5468750	Prec@1 0.007
0:00:24.695843 Epoch: [1][10/102]	Loss 5.5324219	Prec@1 0.008
0:00:25.862809 Epoch: [1][11/102]	Loss 5.5422585	Prec@1 0.007
0:00:26.466211 Epoch: [1][12/102]	Loss 5.5410156	Prec@1 0.007
0:00:30.703127 Epoch: [1][13/102]	Loss 5.5291466	Prec@1 0.006
0:00:31.297017 Epoch: [1][14/102]	Loss 5.5195312	Prec@1 0.007
0:00:33.894129 Epoch: [1][15/102]	Loss 5.5106771	Prec@1 0.006
0:00:34.460986 Epoch: [1][16/102]	Loss 5.5100098	Prec@1 0.006
0

## Predict

In [7]:
model = timm.create_model('tf_efficientnetv2_s.in21k', pretrained=True)
model.reset_classifier(196)

args = Dict()
args.device = "cuda"
args.img_size = 320
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_cv2.pth"
args.output_file = "/home/jupyter/datasphere/project/models/efficientnetv2_cv2_sub.csv"

test_dataset = CarDataset(df_filename="/home/jupyter/mnt/datasets/intro-dl-2024/intro-dl-2024/sample_submission.csv",
                          phase="test",
                          dataroot="/home/jupyter/mnt/datasets/intro-dl-2024/intro-dl-2024/",
                          img_size=args.img_size)

test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=4)

model.load_state_dict(torch.load(args.model_path))
model = model.to(args.device)
model.eval()

predictions = []
with torch.no_grad():
    for features, _, in tqdm(test_loader):
        outputs = model(features.to(args.device))
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

df = pd.read_csv('/home/jupyter/mnt/datasets/intro-dl-2024/intro-dl-2024/sample_submission.csv')
df['label'] = predictions
df.to_csv(args.output_file, index=False)


100%|██████████| 126/126 [00:51<00:00,  2.45it/s]


## Save CV

In [18]:
model = timm.create_model('tf_efficientnetv2_s.in21k', pretrained=True)
model.reset_classifier(196)

args = Dict()
args.device = "cuda"
args.img_size = 320
args.model_path = "/home/jupyter/datasphere/project/models/efficientnetv2_cv2.pth"

test_dataset = CarDataset(df_filename="/home/jupyter/mnt/datasets/intro-dl-2024/intro-dl-2024/sample_submission.csv",
                          phase="test",
                          dataroot="/home/jupyter/mnt/datasets/intro-dl-2024/intro-dl-2024/",
                          img_size=args.img_size)

test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=4)

model.load_state_dict(torch.load(args.model_path))
model = model.to(args.device)
model.eval()

all_outputs = []
with torch.no_grad():
    for features, _ in tqdm(test_loader):
        outputs = model(features.to(args.device))
        all_outputs.extend(outputs.cpu())

tensor_outputs = torch.stack(all_outputs)

100%|██████████| 126/126 [00:49<00:00,  2.55it/s]


In [19]:
tensor_outputs.shape

torch.Size([8041, 196])

In [20]:
torch.save(tensor_outputs, '/home/jupyter/datasphere/project/cv/output_test_2.pt')